# Inner workings of HyperSpy signals

This notebooks shows some of the inner working of HyperSpy, and how to interact directly with the underlying data.

Intended for people who are not familiar with scientific python.

## Author
2017/11/01 Magnus Nord

## Importing library and loading data

In [ ]:
%matplotlib nbagg
import hyperspy.api as hs
s = hs.load("electron_microscopy/EELS/datasets/LSMO_STO_linescan.hdf5")

## Signal object and data

Data-wise the signals can roughly be divided into three separate parts:

- Raw data numbers, contained in `s.data` as a NumPy array
- Axes information, in `s.axes_manager`, like scaling, offset, units, ...
- Metadata, in `s.metadata` instrument information, markers, operator, ...

## The data

All the data is contained within an NumPy array, with the same size as the HyperSpy signal 

In [ ]:
s.data

This NumPy array can be changed directly

In [ ]:
s.data = s.data/10
s.data

## Interlude: scientific Python

Scientific Python is a large group of libraries, which covers everything from large scale array operations, plotting, fitting, image analysis, ...

The most important ones are:

- NumPy: very fast array operations ++
- Matplotlib: plotting and visualization
- SciPy: fitting, image processing, interpolation, ...

All these libraries easily interfaces with eachother, so matplotlib can directly plot at NumPy array, and SciPy can directly work with NumPy arrays.

HyperSpy relies on all of these libraries (and more!), and this tight integration between them makes it very easy to apply many data processing techniques which hasn't been directly implemented in HyperSpy.

For example:

In [ ]:
from scipy.ndimage import gaussian_filter1d
data = s.data[5]
data_blur = gaussian_filter1d(input=data, sigma=3)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(data)
ax.plot(data_blur)

In [ ]:
plt.close()

## Axes manager

All information about the different dimensions are contained in the `axes_manager` object.

In [ ]:
s.axes_manager

Axes are of two different types: signal or navigation, which affects how the data is plotted and (sometimes) processed.

Changing the axes properties does not change the raw data in `s.data`

These values are changed by directly accessing the axis object. So to change the probe dimension axis:

In [ ]:
probe_axis = s.axes_manager.navigation_axes[0]
probe_axis.scale = 31.5
probe_axis.units = 'Å'
probe_axis.name = 'Probe'
probe_axis.offset = -93
s.axes_manager

In [ ]:
s.plot()

The values themselves in an axis object is accessed by:

In [ ]:
probe_axis.axis

Note that the axes must be linear, meaning that the axis values must be evenly spaced. So in the signal `s`, the probe positions cannot be `[-100, 80, 30, 0]`.

It is also possible to "switch" the navigation and signal axes by using transpose

In [ ]:
s_transpose = s.T
s_transpose

So now we got the energy loss dimension as the navigation axis, which makes it easy to see which elements are located where.

In [ ]:
s_transpose.plot()

The ability to "switch" our navigation and signal axes is a very powerful functionality, since it allows for the data to be "viewed" in different directions. In addition, this new signal `s_transpose` does not use any addition memory, since it uses the same data as `s`, just "viewed" in a different direction.

## Metadata dictionary

Lastly, information about the microscope, acquisition parameters, and similar metadata is located in `s.metadata`

In [ ]:
s.metadata

Some of these experimental parameters are used when modelling EELS edges. They can be changed by directly change the value. Note that all these values can be "tab-completed" so it is easy to navigate this metadata "tree".

In [ ]:
s.metadata.Acquisition_instrument.TEM.Detector.EELS.collection_angle = 25

The metadata also contains the elements added using `s.add_elements`

In [ ]:
s.add_elements(('Ti', 'Mn'))
s.metadata.Sample

## Slicing: inav and isig

HyperSpy has extensive support for slicing, which essentially means cropping the data along one or several dimensions. 

The main functions are `s.inav` and `s.isig`:

- `inav` slices in the navigation dimension
- `isig` slices in the signal dimension

If integers are used, it will slice in based on index. If decimal numbers are used, it will slice based on the scaling.

To get just the Ti-L32 edge. (Note the use of decimal points.)

In [ ]:
s_crop = s.isig[400.:500.]
s_crop.plot()

To grab the 20 first pixels in the EELS spectrum.

In [ ]:
s_crop2 = s.isig[0:20]
s_crop2.plot()

An important thing to keep in mind is that the sliced signal `s_crop` uses the same data as `s`, so changing `s_crop` will also change `s`

In [ ]:
s_crop.data /= 2
s.plot()

If want want to get a new copy of the signal, use `s.deepcopy()`.

In [ ]:
s_copy = s.deepcopy()

## HyperSpy and functions

An important part of the HyperSpy signals are the methods in each signal. We've already seen some of them: `plot`, `create_model`, ... There are very many of these methods, and the easiest way to see them all is using tab-completion: in this case, typing the signal variable, add a `.`, and then press the __TAB__ button. 

In [ ]:
s.

The types of functions available will depend on which type of type the signal is. For example our current signal is an EELSSpectrum, which contains methods for aligning the zero loss peak, making EELS models and other similar utility functions.

## Signal types and methods

In addition to the `EELSSpectrum` signal type we've been working with this far, there are several different signal types: `Signal1D`, `Signal2D`, `EDSTEMSpectrum`, `EDSSEMSpectrum`, `HologramImage` and more.

It is possible to convert between the different types, so for our signal `s`

In [ ]:
s_2d = s.as_signal2D((0, 1))

In [ ]:
s_2d.plot()